In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from nbc_analysis.utils.config_utils import get_config
from pathlib import Path
from toolz import take, partial
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from collections import defaultdict
import itertools
from pandas.util import hash_array, hash_pandas_object

In [3]:
CONFIG = get_config(config='default')
SOURCE_D = Path(CONFIG['WORK_D']) / 'query'
WORK_D = Path(CONFIG['WORK_D']) / 'ip_model'
CONFIG

2019-10-18 19:07:39,567|WARNING|nbc_analysis.utils.config_utils|The 'default' config is intended for testing only. For production, use /Users/wmcabee/.config/nbc_analysis/config.yaml


{'CALENDAR_START_DAY_KEY': '20170101',
 'CALENDAR_END_DAY_KEY': '20210101',
 'CALENDAR_D': '/Users/wmcabee/DATA/NBC/calendar',
 'ANALYSIS_D': '/Users/wmcabee/DATA/NBC/ana',
 'RUNS_D': '/Users/wmcabee/DATA/NBC/runs',
 'WORK_D': '/Users/wmcabee/DATA/NBC/work',
 'VIDEO_END_BUCKET': 'nbc-event',
 'VIDEO_END_PARTITIONS_BUCKET': 'nbc-partitions-video-end-protect',
 'GEN_PROFILE_SOURCE': 'nbc-partitions-video-end',
 'BATCH_SIZE': 200000000,
 'GEOLITE2_DB': '/Users/wmcabee/DATA/NBC/datasets/GeoLite2-City_20191001/GeoLite2-City.mmdb',
 'VIEWER_SOURCE_BUCKET': 'nbc-events-partitioned',
 'VIEWER_PARTITION_COUNT': 60,
 'VIEWER_PARTITION_D': '/Users/wmcabee/DATA/NBC/viewer_partitions',
 'LIMIT_FILES_PER_DAY': 20,
 'BATCH_LIMIT': 2,
 'BATCH_FILES_LIMIT': 2,
 'MERGE_LIMIT': 2,
 'PROFILE_PER_WEEK_LIMIT': 1000}

In [4]:
reader = SOURCE_D.glob('*')
reader = take(3, reader)
df = pd.concat(map(pd.read_parquet, reader))
DATA = df.sort_values('event_start_unixtime_ms', ascending=False).reset_index(drop=True)
DATA.head(3).T

,0,1,2
batch_id,ve_20190930_0003,ve_20191001_0004,ve_20191001_0004
file,NBC_App_20190930033852328226_68559.txt,NBC_20191001045954961424_64260.txt,NBC_App_20191001045930779268_51913.txt
file_idx,1532,2017,2015
asof_dt,2019-10-11T19:16:54.139463Z,2019-10-11T19:36:20.621381Z,2019-10-11T19:36:20.621381Z
mpid,7752839992563167640,1074643952692953660,-7141178299446626700
nbc_profile,Unauthenticated,Unauthenticated,Unauthenticated
mvpd,Unauthenticated,DTV,Unauthenticated
event_name,Video End,Video End,Video End
event_type,custom_event,custom_event,custom_event
platform,Not Set,tvOS,Not Set


In [5]:
def build_video_dim(data):
    
    ts_col = 'event_start_unixtime_ms'
    nk ='video_id'
    cols = ['video_type','show','season','episode_number','episode_title','genre','video_duration']

    replace_null_with = 'Not Set'

    ucols  = [nk] + cols
    dcols = ucols + [ts_col]
    

    df = data[data.video_id.notnull()]  # filter out records where video_id is null
    
    df = df[dcols].drop_duplicates(nk)
    df = df.fillna(replace_null_with)
    dim = df

    assert len(dim) == dim[nk].nunique(), "mismatch: %s != %s" % (len(dim), dim[nk].nunique())
    data.drop(columns=cols, inplace=True)
    data[nk] = data.pop(nk)
    return dim, data


In [22]:
DEFAULT_NA ='Not Set'

def build_hash_dim(data, hash_code, cols, upd_ts, fill_na=':ALL', na_val=DEFAULT_NA):

    dcols = [hash_code] + cols
    data[hash_code] = hash_pandas_object(data[cols], index=False).values
    dim = data[dcols].drop_duplicates().sort_values(cols)
    
    data.drop(columns=cols, inplace=True)
    if fill_na:
        if fill_na == ':ALL':
            dim[cols] = dim[cols].fillna(na_val)
        else:
            dim[fill_na] = dim[fill_na].fillna(na_val)
    return dim, data

upd_ts = 'event_start_unixtime_ms'

build_event_type_dim = partial(build_hash_dim, hash_code='event_hash', cols=['event_name', 'event_type'], upd_ts=upd_ts)
build_platform_dim = partial(build_hash_dim, hash_code='platform_hash', cols=['platform','data_connection_type'], upd_ts=upd_ts)
build_profile_dim = partial(build_hash_dim, hash_code='profile_hash', cols=['mvpd','nbc_profile'], upd_ts=upd_ts)
build_end_type_dim = partial(build_hash_dim, hash_code='end_type_hash', cols=['video_end_type','resume'], upd_ts=upd_ts)

In [23]:
def build_unique_set(data, cols):
    values = data[cols].drop_duplicates().sort_values(cols)
    for field in cols:
        data[field] = data.pop(field)
    return values, data

build_ip_set = partial(build_unique_set, cols=['ip'])

In [24]:
def clean_fact(data):
    treat_as_null = {'None','Not Set', ''}


    data = data.mask(data.isin(treat_as_null)) # Replace all null representations with np.NaN
    data['mpid'] = data.pop('mpid')

    data.drop(columns=['session_start_unixtime_ms', 'session_id'], inplace=True)
    
    return data

In [25]:
def set_ts_fields(data):
    
    ts = data['event_start_unixtime_ms'].astype(np.int)
    ts = ts / 1000
    ts = pd.to_datetime(ts, unit='s', origin='unix')

    data['_asof_dt'] = data.pop('asof_dt')
    data['_last_upd_dt'] =np.datetime_as_string(ts, timezone='UTC',unit='us' )
    data['day_utc_key'] = ts.dt.strftime('%Y%m%d')
    dim = data['day_utc_key'].drop_duplicates
    return dim, data



In [26]:
data = DATA.copy()
data = clean_fact(data)
days_dim, data = set_ts_fields(data)


ip_set, data = build_ip_set(data)
end_type_dim, data = build_end_type_dim(data)
platform_dim, data = build_platform_dim(data)
event_type_dim, data = build_event_type_dim(data)
profile_dim, data = build_profile_dim(data)
video_dim, data = build_video_dim(data)
data = data.reset_index(drop=True)

data.head().T

,0,1,2,3,4
batch_id,ve_20190930_0003,ve_20191001_0004,ve_20191001_0004,ve_20191001_0004,ve_20191001_0004
file,NBC_App_20190930033852328226_68559.txt,NBC_20191001045954961424_64260.txt,NBC_App_20191001045930779268_51913.txt,NBC_20191001045915679915_21786.txt,NBC_App_20191001045937759921_60697.txt
file_idx,1532,2017,2015,2012,2016
event_id,-2165054904579433498,-7801022097402524755,-7502478291025245391,-4125335981896278535,-5859414957082708798
video_duration_watched,0.766666666666667,22,0,2,2.11666666666667
event_start_unixtime_ms,1569987267779,1569905923977,1569905885966,1569905870180,1569905870172
resume_time,NaN,NaN,NaN,NaN,NaN
viewer_partition,0,0,0,0,0
mpid,7752839992563167640,1074643952692953660,-7141178299446626700,-2327248636060088100,-7141178299446626700
_asof_dt,2019-10-11T19:16:54.139463Z,2019-10-11T19:36:20.621381Z,2019-10-11T19:36:20.621381Z,2019-10-11T19:36:20.621381Z,2019-10-11T19:36:20.621381Z


In [31]:
dim = video_dim
print('len:', len(dim))
dim

len: 11472


,video_id,video_type,show,season,episode_number,episode_title,genre,video_duration,event_start_unixtime_ms
0,4036451,Highlight,Chicago P.D.,7,1,You Went Behind My Back,Drama,1,1569987267779
1,3675825,Full Episode,A.P. Bio,1,5,Dating Toledoans,Comedy,22,1569905923977
2,4037285,Current Preview,Saturday Night Live,45,1,Saturday Is a Big Night for Woody Harrelson an...,Comedy,1,1569905885966
3,4001075,Full Episode,Chicago P.D.,6,1,New Normal,Drama,41,1569905870180
4,4039074,Highlight,Saturday Night Live,45,1,Weekend Update: Disney World’s Vegan Menu,Comedy,2,1569905870172
...,...,...,...,...,...,...,...,...,...
239395,3688923,Full Episode,Will & Grace,1,15,One Job,Comedy,22,1536380862782
239396,3783035,Full Episode,Days of our Lives,53,235,"Monday, August 27, 2018",Drama,37,1535447009628
239398,3776487,Full Episode,NBC Nightly News,2018,240,"NBC Nightly News, Aug 24, 2018",News and Information,21,1535242752057
239400,3768292,Full Episode,America's Got Talent,13,10,Judge Cuts 3,Reality and Game Show,84,1533167971741


In [12]:
data['end_type_hash'].value_counts()

1543738608339601720     77999
14134703689083528527    63491
10155504590337046581    47014
7968799331887820946     39849
15654977777582709682     5658
3037280689985078357      4778
13058633064681211906      259
14279567677105438984      218
3756609792117231457        89
6152596211329002182        39
6132961493477577762        29
17510052516783454175        1
Name: end_type_hash, dtype: int64